# Simulate data for synergistic algorithms

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib notebook

In [ ]:
import sirf.STIR as pet
import matplotlib.pyplot as plt
import os
import numpy as np

# Get to correct directory
os.chdir("/data/SRS_data_exhale")

# We'll need a template sinogram
templ_sino = pet.AcquisitionData('mMR_template_span11.hs')

In [ ]:
#%% some handy function definitions
def imshow(image, limits=None, title=''):
    """Usage: imshow(image, [min,max], title)"""
    plt.title(title)
    bitmap = plt.imshow(image)
    if limits is None:
        limits = [image.min(), image.max()]
                
    plt.clim(limits[0], limits[1])
    plt.colorbar(shrink=.6)
    plt.axis('off')
    return bitmap

## PET ground truth

In [ ]:
gt_act = pet.ImageData('PET_activity.nii')
gt_atten = pet.ImageData('PET_attenuation.nii')

fig, axs = plt.subplots(2,3);
fig.suptitle('Ground truth PET');

# PET Activity
axs[0,0].set_ylabel('Activity', rotation=90, size='large');
axs[0,0].imshow(gt_act.as_array()[:,:,60]);
axs[0,1].imshow(gt_act.as_array()[:,60,:]);
axs[0,2].imshow(gt_act.as_array()[60,:,:]);

# PET attenuation
axs[1,0].set_ylabel('Attenuation', rotation=90, size='large');
axs[1,0].imshow(gt_atten.as_array()[:,:,60]);
axs[1,1].imshow(gt_atten.as_array()[:,60,:]);
axs[1,2].imshow(gt_atten.as_array()[60,:,:]);

## MR ground truth

In [ ]:
gt_T1 = pet.ImageData('MR_T1.nii')
gt_T2 = pet.ImageData('MR_T2.nii')
gt_PD = pet.ImageData('MR_PD.nii')

fig, axs = plt.subplots(3,3);
fig.suptitle('Ground truth MR');

# MR T1
axs[0,0].set_ylabel('T1', rotation=90, size='large');
axs[0,0].imshow(gt_T1.as_array()[:,:,60]);
axs[0,1].imshow(gt_T1.as_array()[:,60,:]);
axs[0,2].imshow(gt_T1.as_array()[60,:,:]);

# MR T1
axs[1,0].set_ylabel('T2', rotation=90, size='large');
axs[1,0].imshow(gt_T2.as_array()[:,:,60]);
axs[1,1].imshow(gt_T2.as_array()[:,60,:]);
axs[1,2].imshow(gt_T2.as_array()[60,:,:]);

# MR PD
axs[2,0].set_ylabel('PD', rotation=90, size='large');
axs[2,0].imshow(gt_PD.as_array()[:,:,60]);
axs[2,1].imshow(gt_PD.as_array()[:,60,:]);
axs[2,2].imshow(gt_PD.as_array()[60,:,:]);

## Create noisy PET data

In [ ]:
#%% create acquisition model
am = pet.AcquisitionModelUsingRayTracingMatrix()
am.set_num_tangential_LORs(5)

# Set up sensitivity due to attenuation
asm_attn = pet.AcquisitionSensitivityModel(gt_atten, am)
asm_attn.set_up(templ_sino)
bin_eff = pet.AcquisitionData(templ_sino)
bin_eff.fill(1.0)
print('applying attenuation (please wait, may take a while)...')
asm_attn.unnormalise(bin_eff)
asm_attn = pet.AcquisitionSensitivityModel(bin_eff)

am.set_acquisition_sensitivity(asm_attn)

am.set_up(templ_sino,gt_act);

In [ ]:
#%% simulate some data using forward projection
gt_sino = am.forward(gt_act)

gt_sino_array = np.abs(gt_sino.as_array());
noisy_array = np.random.poisson(gt_sino_array).astype('float64');
noisy_sino = gt_sino.clone();
noisy_sino.fill(noisy_array);

### Display it

In [ ]:
sino_max = gt_sino_array[0,400,:,:].max()

plt.figure()
plt.subplot(1,2,1);
imshow(gt_sino_array[0,400,:,:], [0,sino_max], 'Original');
plt.subplot(1,2,2);
imshow(noisy_array[0,400,:,:], [0,sino_max], 'Noisy');

### Save it

In [ ]:
#%% create objective function
noisy_sino.write('noisy_sino')